# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
!pip install gmaps
import gmaps
import os

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,75.56,84,98,8.72,PN,1707194188
1,1,grytviken,-54.2811,-36.5092,43.97,92,40,16.40,GS,1707193904
2,2,port-aux-francais,-49.3500,70.2167,42.58,74,100,28.32,TF,1707194196
3,3,orkney,-26.9802,26.6727,68.72,24,0,1.88,ZA,1707194580
4,4,kiama,-34.6833,150.8667,69.01,92,100,5.01,AU,1707194580


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
gmaps.configure(geoapify_key)

# Display the map
locations = city_data_df[["Lat", "Lng"]]
weights = city_data_df['Humidity']
fig = gmaps.figure(center=(20.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

fig

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_name = []


# Drop any rows with null values
for index, row in city_data_df.iterrows():
    if row["Cloudiness"] == 0:
        if row ["Wind Speed"] < 10:
            if row["Max Temp"] >70 and row["Max Temp"]<90:
                city_name.append(row)

# Display sample data
city_weather_df = pd.DataFrame(city_name)

city_weather_df.to_csv("city_perfect_weather.csv")
city_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,bonthe,7.5264,-12.5050,75.49,91,0,2.48,SL,1707194229
30,30,chipinge,-20.1883,32.6236,72.23,65,0,1.12,ZW,1707194586
32,32,lazaro cardenas,17.9583,-102.2000,73.24,86,0,3.87,MX,1707194586
76,76,vikarabad,17.3333,77.9000,83.52,41,0,7.81,IN,1707194597
101,101,bubaque,11.2833,-15.8333,78.96,59,0,9.04,GW,1707194199


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = [None] * len(hotel_df)

# Display sample data
print(hotel_df.head())

   City_ID               City      Lat       Lng  Max Temp  Humidity  \
0        0          adamstown -25.0660 -130.1015     75.56        84   
1        1          grytviken -54.2811  -36.5092     43.97        92   
2        2  port-aux-francais -49.3500   70.2167     42.58        74   
3        3             orkney -26.9802   26.6727     68.72        24   
4        4              kiama -34.6833  150.8667     69.01        92   

   Cloudiness  Wind Speed Country        Date Hotel Name  
0          98        8.72      PN  1707194188       None  
1          40       16.40      GS  1707193904       None  
2         100       28.32      TF  1707194196       None  
3           0        1.88      ZA  1707194580       None  
4         100        5.01      AU  1707194580       None  


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "filter": "hotel",
    "bias": "[latitude],[longitude]",
    "radius": radius
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = "hotel"
    params["bias"] = f"{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
orkney - nearest hotel: No hotel found
kiama - nearest hotel: No hotel found
sitka - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
bochum - nearest hotel: No hotel found
margate - nearest hotel: No hotel found
saudarkrokur - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
kenai - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
saint-philippe - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel fou

villa gesell - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
chibuto - nearest hotel: No hotel found
aguada de pasajeros - nearest hotel: No hotel found
namibe - nearest hotel: No hotel found
dudinka - nearest hotel: No hotel found
yarkovo - nearest hotel: No hotel found
ternate - nearest hotel: No hotel found
port macquarie - nearest hotel: No hotel found
inda silase - nearest hotel: No hotel found
mahibadhoo - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
east end - nearest hotel: No hotel found
vila franca do campo - nearest hotel: No hotel found
horta - nearest hotel: No hotel found
mar del plata - nearest hotel: No hotel found
naze - nearest hotel: No hotel found
olyokminsk - nearest hotel: No hotel found
meulaboh - nearest hotel: No hotel found
konstantinovka - nearest hotel: No hotel found
antofagasta - nearest hotel: No hotel found
shingu - neare

kodinsk - nearest hotel: No hotel found
grootfontein - nearest hotel: No hotel found
klaksvik - nearest hotel: No hotel found
cidreira - nearest hotel: No hotel found
mossendjo - nearest hotel: No hotel found
north platte - nearest hotel: No hotel found
necochea - nearest hotel: No hotel found
emerald - nearest hotel: No hotel found
auchi - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
dakoro - nearest hotel: No hotel found
macheng - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
puerto aysen - nearest hotel: No hotel found
hanover - nearest hotel: No hotel found
pemba - nearest hotel: No hotel found
houmt el souk - nearest hotel: No hotel found
jaciara - nearest hotel: No hotel found
nguigmi - nearest hotel: No hotel found
kapuskasing - nearest hotel: No hotel found
monforte de lemos - nearest hotel: No hotel found
guanica - nearest hotel: No hotel found
hameln - nearest hotel: No hotel found
georgiyevka - nearest hotel: No hotel found

fort st. john - nearest hotel: No hotel found
kindu - nearest hotel: No hotel found
visakhapatnam - nearest hotel: No hotel found
itaituba - nearest hotel: No hotel found
morant bay - nearest hotel: No hotel found
saryozek - nearest hotel: No hotel found
uman - nearest hotel: No hotel found
weihai - nearest hotel: No hotel found
mandiana - nearest hotel: No hotel found
terrace - nearest hotel: No hotel found
katsuren-haebaru - nearest hotel: No hotel found
cockburn town - nearest hotel: No hotel found
antsiranana - nearest hotel: No hotel found
kaiapoi - nearest hotel: No hotel found
afif - nearest hotel: No hotel found
parauapebas - nearest hotel: No hotel found
port-gentil - nearest hotel: No hotel found
kolwezi - nearest hotel: No hotel found
nova vicosa - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,adamstown,-25.0660,-130.1015,75.56,84,98,8.72,PN,1707194188,No hotel found
1,1,grytviken,-54.2811,-36.5092,43.97,92,40,16.40,GS,1707193904,No hotel found
2,2,port-aux-francais,-49.3500,70.2167,42.58,74,100,28.32,TF,1707194196,No hotel found
3,3,orkney,-26.9802,26.6727,68.72,24,0,1.88,ZA,1707194580,No hotel found
4,4,kiama,-34.6833,150.8667,69.01,92,100,5.01,AU,1707194580,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
600,600,parauapebas,-6.0675,-49.9022,72.55,98,93,0.94,BR,1707194741,No hotel found
601,601,port-gentil,-0.7193,8.7815,81.88,81,99,8.52,GA,1707194344,No hotel found
602,602,kolwezi,-10.7148,25.4667,62.64,98,100,5.64,CD,1707194742,No hotel found
603,603,nova vicosa,-17.8919,-39.3719,76.87,89,98,5.10,BR,1707194287,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Display the map
locations = hotel_df[["Lat", "Lng"]]
weights = hotel_df['Humidity']
fig = gmaps.figure(center=(20.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

fig

AttributeError: module 'collections' has no attribute 'Iterable'